## Analysis number of partient has pair of image L of MLO, CC equal {0, 1}, {1, 0}

In [50]:
import cv2
from PIL import Image, ImageOps
import argparse
import os
import glob
import matplotlib.pyplot as plt
import multiprocessing as mp
from joblib import Parallel, delayed
import pydicom
import time
import numpy as np
import torch
import random
import pandas as pd
from pydicom.pixel_data_handlers import apply_windowing
import imageio
import time
import shutil

In [51]:
meta = pd.read_csv(r"E:\breast-level_annotations-1.csv")
meta

,study_id,series_id,image_id,laterality,view_position,height,width,breast_birads,breast_density,split
0,b8d273e8601f348d3664778dae0e7e0b,b36517b9cbbcfd286a7ae04f643af97a,d8125545210c08e1b1793a5af6458ee2,L,CC,3518,2800,BI-RADS 2,DENSITY C,training
1,b8d273e8601f348d3664778dae0e7e0b,b36517b9cbbcfd286a7ae04f643af97a,290c658f4e75a3f83ec78a847414297c,L,MLO,3518,2800,BI-RADS 2,DENSITY C,training
2,b8d273e8601f348d3664778dae0e7e0b,b36517b9cbbcfd286a7ae04f643af97a,cd0fc7bc53ac632a11643ac4cc91002a,R,CC,3518,2800,BI-RADS 2,DENSITY C,training
3,b8d273e8601f348d3664778dae0e7e0b,b36517b9cbbcfd286a7ae04f643af97a,71638b1e853799f227492bfb08a01491,R,MLO,3518,2800,BI-RADS 2,DENSITY C,training
4,8269f5971eaca3e5d3772d1796e6bd7a,d931832a0815df082c085b6e09d20aac,dd9ce3288c0773e006a294188aadba8e,L,CC,3518,2800,BI-RADS 1,DENSITY C,training
...,...,...,...,...,...,...,...,...,...,...
19995,f2093a752e6b44df5990f5fd38c99dd2,2b1b2b8f48abab9819c0b3d091e152ee,ea732154d149f619b20070b78060ae65,R,CC,2812,2012,BI-RADS 2,DENSITY C,training
19996,b3c8969cd2accfa4dbb2aece1f7158ab,69d7f07ea04572dad5e5aa62fbcfc4b7,4689616c3d0b46fcba7a771107730791,R,CC,3580,2702,BI-RADS 2,DENSITY C,training
19997,b3c8969cd2accfa4dbb2aece1f7158ab,69d7f07ea04572dad5e5aa62fbcfc4b7,3c22491bcf1d0b004715c28d80981cdd,L,CC,3580,2702,BI-RADS 2,DENSITY C,training
19998,b3c8969cd2accfa4dbb2aece1f7158ab,69d7f07ea04572dad5e5aa62fbcfc4b7,d443b9725e331b8b27589aa725597801,R,MLO,3580,2686,BI-RADS 2,DENSITY C,training


In [52]:
meta['breast_birads'] = meta.breast_birads.apply(lambda x: int(x[-1]))

In [53]:
meta = meta[['image_id', 'breast_birads', 'split']]
meta.head(5)

,image_id,breast_birads,split
0,d8125545210c08e1b1793a5af6458ee2,2,training
1,290c658f4e75a3f83ec78a847414297c,2,training
2,cd0fc7bc53ac632a11643ac4cc91002a,2,training
3,71638b1e853799f227492bfb08a01491,2,training
4,dd9ce3288c0773e006a294188aadba8e,1,training


In [54]:
meta.split.value_counts()

split
training    16000
test         4000
Name: count, dtype: int64

In [55]:
train = meta[meta.split == 'training'].reset_index().drop(['index', 'split'], axis=1)
train

,image_id,breast_birads
0,d8125545210c08e1b1793a5af6458ee2,2
1,290c658f4e75a3f83ec78a847414297c,2
2,cd0fc7bc53ac632a11643ac4cc91002a,2
3,71638b1e853799f227492bfb08a01491,2
4,dd9ce3288c0773e006a294188aadba8e,1
...,...,...
15995,ea732154d149f619b20070b78060ae65,2
15996,4689616c3d0b46fcba7a771107730791,2
15997,3c22491bcf1d0b004715c28d80981cdd,2
15998,d443b9725e331b8b27589aa725597801,2


In [56]:
test = meta[meta.split == 'test'].reset_index().drop(['index', 'split'], axis=1)
test

,image_id,breast_birads
0,04c9b9305ae9a552975cceab2c15632e,1
1,a0b73dcf1c300988780abb9abe55826d,1
2,8132013f5158a2bc93d77ea9200c75d8,2
3,09c140f91b2da7c7ff024e5f4b2d5599,2
4,b045e8b4cbcacce80adb90ee8a9f885a,1
...,...,...
3995,4adfc32f03228403f518f24e9b9313cf,1
3996,4835a79d7e25eebd20104e7147cbcfb4,2
3997,ebb437593a3fc80277296638f0607f5b,2
3998,9188a01b6b648cee39fb70702a860141,1


In [41]:
# cdd = pd.read_csv(r"E:\BB\meta.csv")
# cdd.head(5)

,Unnamed: 0,Image_name,BIRADS
0,0,P1_L_DM_MLO,4
1,1,P2_L_DM_CC,2
2,2,P2_L_DM_MLO,2
3,3,P3_L_DM_CC,5
4,4,P3_L_DM_MLO,5


In [43]:
# cdd = cdd.rename(columns={'Image_name' : 'image_id', "BIRADS": "breast_birads"}).drop('Unnamed: 0', axis=1)
# cdd

,image_id,breast_birads
0,P1_L_DM_MLO,4
1,P2_L_DM_CC,2
2,P2_L_DM_MLO,2
3,P3_L_DM_CC,5
4,P3_L_DM_MLO,5
...,...,...
927,P325_L_DM_MLO,1
928,P326_R_DM_CC,2
929,P326_R_DM_MLO,2
930,P326_L_DM_CC,3


In [35]:
# cdd

In [44]:
# train = pd.concat([train, cdd]).reset_index()
# train

,index,image_id,breast_birads
0,0,d8125545210c08e1b1793a5af6458ee2,2
1,1,290c658f4e75a3f83ec78a847414297c,2
2,2,cd0fc7bc53ac632a11643ac4cc91002a,2
3,3,71638b1e853799f227492bfb08a01491,2
4,4,dd9ce3288c0773e006a294188aadba8e,1
...,...,...,...
16927,927,P325_L_DM_MLO,1
16928,928,P326_R_DM_CC,2
16929,929,P326_R_DM_MLO,2
16930,930,P326_L_DM_CC,3


In [45]:
train.drop('index', axis=1, inplace=True)

In [57]:
train

,image_id,breast_birads
0,d8125545210c08e1b1793a5af6458ee2,2
1,290c658f4e75a3f83ec78a847414297c,2
2,cd0fc7bc53ac632a11643ac4cc91002a,2
3,71638b1e853799f227492bfb08a01491,2
4,dd9ce3288c0773e006a294188aadba8e,1
...,...,...
15995,ea732154d149f619b20070b78060ae65,2
15996,4689616c3d0b46fcba7a771107730791,2
15997,3c22491bcf1d0b004715c28d80981cdd,2
15998,d443b9725e331b8b27589aa725597801,2


In [58]:
train.to_csv(r'E:\WORKBASE\WBB\Vindr\train.csv')

In [59]:
test

,image_id,breast_birads
0,04c9b9305ae9a552975cceab2c15632e,1
1,a0b73dcf1c300988780abb9abe55826d,1
2,8132013f5158a2bc93d77ea9200c75d8,2
3,09c140f91b2da7c7ff024e5f4b2d5599,2
4,b045e8b4cbcacce80adb90ee8a9f885a,1
...,...,...
3995,4adfc32f03228403f518f24e9b9313cf,1
3996,4835a79d7e25eebd20104e7147cbcfb4,2
3997,ebb437593a3fc80277296638f0607f5b,2
3998,9188a01b6b648cee39fb70702a860141,1


In [49]:
test.to_csv(r'E:\WORKBASE\WBB\Vindr\test.csv')

# Concat datasets

In [62]:
pd.read_excel

<function pandas.io.excel._base.read_excel(io, sheet_name: 'str | int | list[IntStrT] | None' = 0, *, header: 'int | Sequence[int] | None' = 0, names: 'list[str] | None' = None, index_col: 'int | Sequence[int] | None' = None, usecols: 'int | str | Sequence[int] | Sequence[str] | Callable[[str], bool] | None' = None, squeeze: 'bool | None' = None, dtype: 'DtypeArg | None' = None, engine: "Literal['xlrd', 'openpyxl', 'odf', 'pyxlsb'] | None" = None, converters: 'dict[str, Callable] | dict[int, Callable] | None' = None, true_values: 'Iterable[Hashable] | None' = None, false_values: 'Iterable[Hashable] | None' = None, skiprows: 'Sequence[int] | int | Callable[[int], object] | None' = None, nrows: 'int | None' = None, na_values=None, keep_default_na: 'bool' = True, na_filter: 'bool' = True, verbose: 'bool' = False, parse_dates: 'list | dict | bool' = False, date_parser: 'Callable | None' = None, thousands: 'str | None' = None, decimal: 'str' = '.', comment: 'str | None' = None, skipfooter: 

In [63]:
rsna = pd.read_csv(r'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\DATA_STANDARD\RSNA.csv')
vindr = pd.read_csv(r'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\DATA_STANDARD\Vindr.csv')
cmmd = pd.read_csv(r'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\DATA_STANDARD\CMMD.csv')
cdd = pd.read_csv(r'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\DATA_STANDARD\cdd_cesm.csv')


In [64]:
cdd = cdd[["patient_id",	"image_id"	,"view",	"laterality"	,	"cancer"]]
cmmd = cmmd[["patient_id",	"image_id"	,"view",	"laterality"	,	"cancer"]]
rsna = rsna[["patient_id",	"image_id"	,"view",	"laterality"	,	"cancer"]]
vindr = vindr[["patient_id",	"image_id"	,"view",	"laterality"	,	"cancer"]]

In [5]:
rsna

,patient_id,image_id,view,laterality,cancer
0,10006,1874946579,CC,R,0
1,10006,1864590858,MLO,R,0
2,10006,462822612,CC,L,0
3,10006,1459541791,MLO,L,0
4,10011,541722628,CC,R,0
...,...,...,...,...,...
49963,9973,1703611570,MLO,L,0
49964,9989,439796429,CC,R,0
49965,9989,398038886,MLO,R,0
49966,9989,1078943060,CC,L,0


In [5]:
# Chia dữ liệu theo cấu trúc VIEW -- > LAT --> IMAGE 

# Convert format new

In [6]:
from remove_image_views import *

In [7]:
rs  = move_image(rsna, 'RSNA')

In [8]:
list(rsna.index)

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [10]:
rs.my_method(list(rsna.index))

In [11]:
len(rsna.index)

49968

In [13]:
rs1  = move_image(vindr, 'Vindr')

In [14]:
rs1.my_method(list(vindr.index))

In [15]:
rs2  = move_image(cmmd, 'CMMD')
rs2.my_method(list(cmmd.index))

In [16]:
rs3  = move_image(cdd, 'CDD-CESM')
rs3.my_method(list(cmmd.index))

KeyError: 638

In [6]:
len(rsna) + len(cmmd) + len(vindr) +  len(cdd)

53750

In [7]:
53750 - 53112

638

In [22]:
list(cdd.index)

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [ ]:
meta = pd.concat()

In [14]:
meta = pd.concat([rsna, vindr, cdd, cmmd]).reset_index()

In [15]:
meta = meta.drop('index', axis=1)
meta

,patient_id,image_id,view,laterality,cancer
0,10006,1874946579,CC,R,0
1,10006,1864590858,MLO,R,0
2,10006,462822612,CC,L,0
3,10006,1459541791,MLO,L,0
4,10011,541722628,CC,R,0
...,...,...,...,...,...
53745,D2-0747,D2-0747-1-3,MLO,R,1
53746,D2-0748,D2-0748-1-4,CC,R,1
53747,D2-0748,D2-0748-1-3,MLO,R,1
53748,D2-0749,D2-0749-1-2,CC,L,1


In [28]:
meta.to_csv('D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Data_main\CSV_MAIN\meta_data.csv')

In [51]:
for name  in cdd.patient_id.unique():
    path = f"D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/Data_main/Dataset_VIEW/{name}"
    try:
        shutil.rmtree(path)
    except:
        print(path)

D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/Data_main/Dataset_VIEW/1CDD
D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/Data_main/Dataset_VIEW/3CDD
D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/Data_main/Dataset_VIEW/7CDD
D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/Data_main/Dataset_VIEW/12CDD
D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/Data_main/Dataset_VIEW/16CDD
D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/Data_main/Dataset_VIEW/18CDD
D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/Data_main/Dataset_VIEW/24CDD
D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/

In [5]:
from  remove_image_views import *
rs3  = move_image(data=cdd, stores='CDD-CESM', crop= False)
rs3.my_method(list(cmmd.index))

KeyError: 638

In [33]:
glob.glob("D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Data_main\Dataset_VIEW\*")

['D:\\OneDrive - Industrial University of HoChiMinh City\\WORKBASE\\Project-rsna-breast-cancer-detection\\Data_main\\Dataset_VIEW\\01c3c13f0b852aed00b5147c21d0e649',
 'D:\\OneDrive - Industrial University of HoChiMinh City\\WORKBASE\\Project-rsna-breast-cancer-detection\\Data_main\\Dataset_VIEW\\0246c6050121c176d2e8fcf91f22f926',
 'D:\\OneDrive - Industrial University of HoChiMinh City\\WORKBASE\\Project-rsna-breast-cancer-detection\\Data_main\\Dataset_VIEW\\0f1a7b0d4efecb388a4b98ae5e2d5c29',
 'D:\\OneDrive - Industrial University of HoChiMinh City\\WORKBASE\\Project-rsna-breast-cancer-detection\\Data_main\\Dataset_VIEW\\0fab8097f55f21d4bfa7667250122312',
 'D:\\OneDrive - Industrial University of HoChiMinh City\\WORKBASE\\Project-rsna-breast-cancer-detection\\Data_main\\Dataset_VIEW\\10006',
 'D:\\OneDrive - Industrial University of HoChiMinh City\\WORKBASE\\Project-rsna-breast-cancer-detection\\Data_main\\Dataset_VIEW\\10011',
 'D:\\OneDrive - Industrial University of HoChiMinh City\\

In [8]:
ls = []

In [9]:
for path in glob.glob("D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Data_main\Dataset_VIEW\*"):
    name = os.path.basename(path)
    # print(name)
    if 'CDD' in name:
        # print("YES")
        # print(name)
        ls.append(name)

In [10]:
len(ls)

179

In [6]:
len(cdd.patient_id.unique())

179

In [12]:
meta

NameError: name 'meta' is not defined

# Chia data into train (70%), test (20), val (10)

# Simple view

In [65]:
from sklearn.model_selection import train_test_split

In [66]:
def save(train, test, val, name = "Rong", name_dataset = "rong"):
    path = f'D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/Data_main/CSV_MAIN/{name_dataset}/{name}'
    os.makedirs(path)
    # train = pd.concat([cancer_train, nocancer_train]).reset_index().drop('index', axis=1)
    # val = pd.concat([cancer_val, nocancer_val]).reset_index().drop('index', axis=1)
    # test = pd.read_csv(r'D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection\DATA_STANDARD\DDSM.csv', index_col='Unnamed: 0')
    ev = Evaluate(train, "Train")
    ev1 = Evaluate(val, "Val")
    ev2 = Evaluate(test, "Test")
    ev.show(f'{path}/description.txt')
    ev1.show(f'{path}/description.txt')
    ev2.show(f'{path}/description.txt')
    train.to_csv(f'D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/Data_main/CSV_MAIN/{name_dataset}/{name}/train.csv')
    val.to_csv(f'D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/Data_main/CSV_MAIN/{name_dataset}/{name}/val.csv')
    test.to_csv(f'D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/Data_main/CSV_MAIN/{name_dataset}/{name}/test.csv')

In [70]:
class Evaluate:
    def __init__(self, meta_pro, name):
        self.al_data = meta_pro.reset_index()
        self.name = name
    def quatity_cc_mlo(self):
        return f"\nSo luong CC và MLO\n{self.al_data.view.value_counts()}\n"
        # print(self.al_data.view.value_counts())
    def lelf_right(self):
        return f"\nSo luong L và R\n{self.al_data.groupby(['view', 'laterality']).size()}\n"
        # print(self.al_data.groupby(['view', 'laterality']).size())
    def show_one(self, par):
        return self.al_data[self.al_data.patient_id == par]
    def quanlity_cancer(self):
        sl = len(self.al_data[self.al_data.cancer == 1])
        return f"So luong img cancer: {sl}"
        # print(self.al_data[self.al_data.cancer == 1])
    def show(self, path):
        ls = len(self.al_data)
        par = len(self.al_data.patient_id.unique())
        # print( 
        # self.quatity_cc_mlo()
        # self.lelf_right()
        # self.quanlity_cancer()
        f = open(path, 'a', encoding='utf-8')
        f.write(f'Dataset {self.name} \n Số lượng image: {ls} \n Số luong patient: {par} {self.quatity_cc_mlo()} {self.lelf_right()} {self.quanlity_cancer()}\n\n +++++++++++++++\n\n')
        f.close()

In [71]:
def pro(data, dup = 1, name = "Rong", name_dataset = "rong"):
    cancer = data[data.cancer == 1].reset_index()
    nocancer = data[data.cancer == 0].reset_index()
    cancer_train, cancer_test_val = train_test_split(cancer, test_size=0.3, random_state=42)
    cancer_test, cancer_val = train_test_split(cancer_test_val, test_size=0.1, random_state=42)
    
    # no_cancer_train, no_cancer_test_val = train_test_split(no_cancer_data, test_size=0.3, random_state=42)
    # no_cancer_test, no_cancer_val = train_test_split(no_cancer_test_val,  test_size=0.1, random_state=42)
    n = (len(cancer_train) + 100)*dup
    # remain = 
    no_cancer_train = nocancer[: n]
    no_cancer_val = nocancer[n: n + len(cancer_val)+100]
    n = n + len(cancer_val)+100
    no_cancer_test = nocancer[n : n + 100 + len(cancer_test)]
    train = pd.concat([cancer_train, no_cancer_train], axis=0).reset_index().drop('index', axis=1)
    test = pd.concat([cancer_test, no_cancer_test], axis=0).reset_index().drop('index', axis=1)
    val = pd.concat([cancer_val, no_cancer_val], axis=0).reset_index().drop('index', axis=1)
    save(train, test, val,name,  name_dataset)
    # print(f'Train: {len(cancer_train)}, Test: {len(cancer_test)}, Val: {len(cancer_val)}')
    # print(f'Train: {len(train)}, Test: {len(test)}, Val: {len(val)}')
    print()
    

In [72]:
pro(rsna, 1, 'rate11', 'RSNA')

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/Data_main/CSV_MAIN/RSNA/rate11'

In [51]:
pro(rsna, 2, 'rate12', 'RSNA')

In [52]:
pro(rsna, 3, 'rate13', 'RSNA')

In [54]:
ddsm = pd.read_csv('D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\DATA_STANDARD\DDSM.csv')

In [74]:
for i in range(1, 4):
    pro(cmmd, i, f'rate1{i}', 'CMMD')

In [32]:
pro(rsna, 3)

Train: 725, Test: 279, Val: 32
Train: 3200, Test: 658, Val: 164


In [11]:
cancer_rsna = rsna[rsna.cancer == 1].reset_index()
nocancer_rsna = rsna[rsna.cancer == 0].reset_index()


In [13]:
cancer_train, cancer_test_val = train_test_split(cancer_data, test_size=0.3, random_state=42)

,index,patient_id,image_id,view,laterality,cancer
0,0,10006,1874946579,CC,R,0
1,1,10006,1864590858,MLO,R,0
2,2,10006,462822612,CC,L,0
3,3,10006,1459541791,MLO,L,0
4,4,10011,541722628,CC,R,0
...,...,...,...,...,...,...
48927,49963,9973,1703611570,MLO,L,0
48928,49964,9989,439796429,CC,R,0
48929,49965,9989,398038886,MLO,R,0
48930,49966,9989,1078943060,CC,L,0


In [7]:
meta = pd.read_csv(r'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Data_main\CSV_MAIN\meta_data.csv')
meta = meta.drop('Unnamed: 0', axis=1)
meta

,patient_id,image_id,view,laterality,cancer
0,10006,1874946579,CC,R,0
1,10006,1864590858,MLO,R,0
2,10006,462822612,CC,L,0
3,10006,1459541791,MLO,L,0
4,10011,541722628,CC,R,0
...,...,...,...,...,...
53745,D2-0747,D2-0747-1-3,MLO,R,1
53746,D2-0748,D2-0748-1-4,CC,R,1
53747,D2-0748,D2-0748-1-3,MLO,R,1
53748,D2-0749,D2-0749-1-2,CC,L,1


In [8]:
meta.image_id = meta.image_id.astype(str)
meta.image_id

0         1874946579
1         1864590858
2          462822612
3         1459541791
4          541722628
            ...     
53745    D2-0747-1-3
53746    D2-0748-1-4
53747    D2-0748-1-3
53748    D2-0749-1-2
53749    D2-0749-1-1
Name: image_id, Length: 53750, dtype: object

In [9]:
meta.loc[50421]

patient_id           1CDD
image_id      P1-L-DM-MLO
view                   CC
laterality              L
cancer                  0
Name: 50421, dtype: object

In [10]:
meta_x = meta.copy()

In [11]:
cancer_data = meta[meta.cancer == 1]
no_cancer_data = meta[meta.cancer == 0]

In [12]:
len(no_cancer_data) + len(cancer_data)

53750

In [13]:
meta.cancer.value_counts()

0    49525
1     4225
Name: cancer, dtype: int64

In [14]:
cancer_data.cancer.value_counts()

1    4225
Name: cancer, dtype: int64

In [15]:
cancer_data

,patient_id,image_id,view,laterality,cancer
82,10130,1360338805,CC,L,1
83,10130,1672636630,MLO,L,1
84,10130,613462606,CC,L,1
85,10130,388811999,MLO,L,1
180,10226,530620473,CC,L,1
...,...,...,...,...,...
53745,D2-0747,D2-0747-1-3,MLO,R,1
53746,D2-0748,D2-0748-1-4,CC,R,1
53747,D2-0748,D2-0748-1-3,MLO,R,1
53748,D2-0749,D2-0749-1-2,CC,L,1


In [16]:
cancer_data.groupby(['laterality', 'view']).count()

patient_id  image_id  cancer
laterality view                              
L          CC          1116      1116    1116
           MLO         1121      1121    1121
R          CC           994       994     994
           MLO          994       994     994

In [17]:
## đếm số lượng trường hợp 11
remain_cancer = cancer_data.copy()
check = {}
for i in cancer_data.image_id:
    check[i] = False
cnt11 = 0
cnt10 = 0
dictview = {
    'CC': 'MLO',
    'MLO': 'CC'
}

for i in cancer_data.index:
    idimg = cancer_data.image_id[i]
    if check[idimg]: continue
    lat = cancer_data.laterality[i]
    View = cancer_data.view[i]
    par = cancer_data.patient_id[i]
    data = remain_cancer[(remain_cancer.laterality == lat) & (remain_cancer.view == dictview[View]) & (remain_cancer.patient_id == par)].index
    # print(data)
    if len(data) > 0:
        try:
            id = remain_cancer.image_id[data[0]]
        
        except:
            print(f'ERRRO: {idimg}, id = {i}')
        cnt11+=1
        check[id] = True
        check[idimg] = True
        remain_cancer = remain_cancer.drop([i, data[0]], axis=0)
    else:
        cnt10 +=1


In [18]:
remain_cancer

,patient_id,image_id,view,laterality,cancer
50420,1CDD,P1-L-DM-MLO,MLO,L,1
50428,7CDD,P7-L-DM-MLO,MLO,L,1
50435,16CDD,P16-R-DM-MLO,MLO,R,1
50452,28CDD,P28-L-DM-MLO,MLO,L,1
50456,29CDD,P29-L-DM-CC,CC,L,1
50462,34CDD,P34-R-DM-MLO,MLO,R,1
50470,47CDD,P47-R-DM-CC,CC,R,1
50476,53CDD,P53-R-DM-CC,CC,R,1
50484,56CDD,P56-L-DM-MLO,MLO,L,1
50490,60CDD,P60-L-DM-MLO,MLO,L,1


In [19]:
2130*2

4260

In [20]:
meta_x

,patient_id,image_id,view,laterality,cancer
0,10006,1874946579,CC,R,0
1,10006,1864590858,MLO,R,0
2,10006,462822612,CC,L,0
3,10006,1459541791,MLO,L,0
4,10011,541722628,CC,R,0
...,...,...,...,...,...
53745,D2-0747,D2-0747-1-3,MLO,R,1
53746,D2-0748,D2-0748-1-4,CC,R,1
53747,D2-0748,D2-0748-1-3,MLO,R,1
53748,D2-0749,D2-0749-1-2,CC,L,1


In [21]:
for i in remain_cancer.index:
    idimg = remain_cancer.image_id[i]
    lat = remain_cancer.laterality[i]
    View = remain_cancer.view[i]
    par = remain_cancer.patient_id[i]
    
    data = no_cancer_data[(no_cancer_data.laterality == lat) & (no_cancer_data.view == dictview[View]) & (no_cancer_data.patient_id == par)].index
    if len(data) > 0:
        pre = meta_x.cancer[data[0]]
        meta_x.cancer[data[0]] = 1
        print(f'Truoc: {pre} Sau: {meta_x.cancer[data[0]]}')
        print(data[0])

C:\Users\nguye\AppData\Local\Temp\ipykernel_3040\3038536961.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meta_x.cancer[data[0]] = 1


Truoc: 0 Sau: 1
50421
Truoc: 0 Sau: 1
50429
Truoc: 0 Sau: 1
50434
Truoc: 0 Sau: 1
50453
Truoc: 0 Sau: 1
50457
Truoc: 0 Sau: 1
50463
Truoc: 0 Sau: 1
50471
Truoc: 0 Sau: 1
50477
Truoc: 0 Sau: 1
50485
Truoc: 0 Sau: 1
50491
Truoc: 0 Sau: 1
50493
Truoc: 0 Sau: 1
50501
Truoc: 0 Sau: 1
50503
Truoc: 0 Sau: 1
50521
Truoc: 0 Sau: 1
50543
Truoc: 0 Sau: 1
50565
Truoc: 0 Sau: 1
50569
Truoc: 0 Sau: 1
50573
Truoc: 0 Sau: 1
50591
Truoc: 0 Sau: 1
50613
Truoc: 0 Sau: 1
50615
Truoc: 0 Sau: 1
50617
Truoc: 0 Sau: 1
50653
Truoc: 0 Sau: 1
50659
Truoc: 0 Sau: 1
50661
Truoc: 0 Sau: 1
50699
Truoc: 0 Sau: 1
50719
Truoc: 0 Sau: 1
50729
Truoc: 0 Sau: 1
50763
Truoc: 0 Sau: 1
50769
Truoc: 0 Sau: 1
50775
Truoc: 0 Sau: 1
50795
Truoc: 0 Sau: 1
50801
Truoc: 0 Sau: 1
50803
Truoc: 0 Sau: 1
50805


In [22]:
meta.loc[50421]

patient_id           1CDD
image_id      P1-L-DM-MLO
view                   CC
laterality              L
cancer                  0
Name: 50421, dtype: object

In [23]:
meta_x.loc[50421]

patient_id           1CDD
image_id      P1-L-DM-MLO
view                   CC
laterality              L
cancer                  1
Name: 50421, dtype: object

In [24]:
meta.cancer.value_counts()

0    49525
1     4225
Name: cancer, dtype: int64

In [25]:
meta_x.cancer.value_counts()

0    49490
1     4260
Name: cancer, dtype: int64

In [29]:
meta_x

,patient_id,image_id,view,laterality,cancer
0,10006,1874946579,CC,R,0
1,10006,1864590858,MLO,R,0
2,10006,462822612,CC,L,0
3,10006,1459541791,MLO,L,0
4,10011,541722628,CC,R,0
...,...,...,...,...,...
53745,D2-0747,D2-0747-1-3,MLO,R,1
53746,D2-0748,D2-0748-1-4,CC,R,1
53747,D2-0748,D2-0748-1-3,MLO,R,1
53748,D2-0749,D2-0749-1-2,CC,L,1


In [30]:
cancer = meta_x[meta_x.cancer == 1].reset_index().drop('index', axis=1)
cancer

,patient_id,image_id,view,laterality,cancer
0,10130,1360338805,CC,L,1
1,10130,1672636630,MLO,L,1
2,10130,613462606,CC,L,1
3,10130,388811999,MLO,L,1
4,10226,530620473,CC,L,1
...,...,...,...,...,...
4255,D2-0747,D2-0747-1-3,MLO,R,1
4256,D2-0748,D2-0748-1-4,CC,R,1
4257,D2-0748,D2-0748-1-3,MLO,R,1
4258,D2-0749,D2-0749-1-2,CC,L,1


In [31]:
s = 0;
cnt = 0

In [50]:
for i in cancer.groupby('patient_id')['image_id'].count():
    if s > 4000:
        break
    cnt+=1
    s+=i

In [51]:
len(cancer.groupby('patient_id')['image_id'].count())

2088

In [52]:
cnt

1960

In [53]:
s

4002

In [54]:
4260 - 4002

258

In [55]:
# Chúng ta sẽ lấy cỡ 1955 bệnh nhân cancer đầu làm train có 4002 image
# còn 133 bệnh nhân và khoảng  268 image cho val
# tập test sẽ là số bênh nhân trong DDSM

In [56]:
par = meta_x[meta_x.cancer == 1].patient_id.unique()
len(par)

2088

In [57]:
cancer

,patient_id,image_id,view,laterality,cancer
0,10130,1360338805,CC,L,1
1,10130,1672636630,MLO,L,1
2,10130,613462606,CC,L,1
3,10130,388811999,MLO,L,1
4,10226,530620473,CC,L,1
...,...,...,...,...,...
4255,D2-0747,D2-0747-1-3,MLO,R,1
4256,D2-0748,D2-0748-1-4,CC,R,1
4257,D2-0748,D2-0748-1-3,MLO,R,1
4258,D2-0749,D2-0749-1-2,CC,L,1


In [58]:
par

array(['10130', '10226', '1025', ..., 'D2-0747', 'D2-0748', 'D2-0749'],
      dtype=object)

In [59]:
cancer_train = cancer[cancer.patient_id.isin(par[:1960])].reset_index().drop('index', axis=1)
cancer_train

,patient_id,image_id,view,laterality,cancer
0,10130,1360338805,CC,L,1
1,10130,1672636630,MLO,L,1
2,10130,613462606,CC,L,1
3,10130,388811999,MLO,L,1
4,10226,530620473,CC,L,1
...,...,...,...,...,...
3997,D2-0619,D2-0619-1-3,MLO,R,1
3998,D2-0620,D2-0620-1-4,CC,R,1
3999,D2-0620,D2-0620-1-3,MLO,R,1
4000,D2-0621,D2-0621-1-2,CC,L,1


In [60]:
cancer_val = cancer[cancer.patient_id.isin(par[1955:])].reset_index().drop('index', axis=1)
cancer_val

,patient_id,image_id,view,laterality,cancer
0,D2-0617,D2-0617-1-4,CC,R,1
1,D2-0617,D2-0617-1-3,MLO,R,1
2,D2-0618,D2-0618-1-2,CC,L,1
3,D2-0618,D2-0618-1-1,MLO,L,1
4,D2-0619,D2-0619-1-4,CC,R,1
...,...,...,...,...,...
263,D2-0747,D2-0747-1-3,MLO,R,1
264,D2-0748,D2-0748-1-4,CC,R,1
265,D2-0748,D2-0748-1-3,MLO,R,1
266,D2-0749,D2-0749-1-2,CC,L,1


In [61]:
# Danh gia lai so luong MLO va CC trong tap cancer

In [62]:
from Evaluate import *

In [63]:
evtrain = Evaluate(cancer_train, 'cancer_train')
evval = Evaluate(cancer_val, 'cancer_val')

In [64]:
evtrain.show()

Dataset cancer_train 
 Số lượng image: 4002 
 Số luong patient: 1960
So luong CC và MLO
CC     2001
MLO    2001
Name: view, dtype: int64
So luong L và R
view  laterality
CC    L             1065
      R              936
MLO   L             1065
      R              936
dtype: int64
So luong img cancer: 4002


In [65]:
evval.show()

Dataset cancer_val 
 Số lượng image: 268 
 Số luong patient: 133
So luong CC và MLO
CC     134
MLO    134
Name: view, dtype: int64
So luong L và R
view  laterality
CC    L             66
      R             68
MLO   L             66
      R             68
dtype: int64
So luong img cancer: 268


In [107]:
def processing_train_nocancer(data, rate):
    level = 4500 * rate
    nocancer = data[data.cancer==0]
    no_par =  nocancer.groupby(['patient_id'])['image_id'].count()
    cnt = 0
    s = 0
    for i in no_par:
        if s > level:
            break
        s+=i
        cnt+=1
    par = no_par.index
    nocancer_train = nocancer[nocancer.patient_id.isin(par[:cnt])].reset_index()
    return nocancer_train[["patient_id",	"image_id",	"view",	"laterality",	"cancer"]]


In [113]:
no_use = pd.read_csv(r'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Data_main\CSV_MAIN\Rate11\no_use.csv')

In [114]:
no_use

,index,patient_id,image_id,view,laterality,cancer
0,4842,1568,1557949832,CC,R,0
1,4843,1568,1737493826,MLO,R,0
2,4844,1568,1729179759,CC,L,0
3,4845,1568,200549961,MLO,L,0
4,4846,15689,915609818,CC,R,0
...,...,...,...,...,...,...
44691,53469,D2-0613,D2-0613-1-3,MLO,R,0
44692,53592,D2-0674,D2-0674-1-4,CC,R,0
44693,53593,D2-0674,D2-0674-1-3,MLO,R,0
44694,53650,D2-0701,D2-0701-1-2,CC,L,0


In [110]:
standard_nocancer_train = processing_train_nocancer(meta_x, 1)

In [111]:
standard_nocancer_train

,patient_id,image_id,view,laterality,cancer
0,10006,1874946579,CC,R,0
1,10006,1864590858,MLO,R,0
2,10006,462822612,CC,L,0
3,10006,1459541791,MLO,L,0
4,10011,541722628,CC,R,0
...,...,...,...,...,...
4499,146CDD,P146-L-DM-MLO,MLO,L,0
4500,149CDD,P149-L-DM-CC,CC,L,0
4501,149CDD,P149-L-DM-MLO,MLO,L,0
4502,150CDD,P150-R-DM-CC,CC,R,0


In [108]:
nocancer_train = processing_train_nocancer(no_use, 2)
nocancer_train

,patient_id,image_id,view,laterality,cancer
0,10006,1874946579,CC,R,0
1,10006,1864590858,MLO,R,0
2,10006,462822612,CC,L,0
3,10006,1459541791,MLO,L,0
4,10011,541722628,CC,R,0
...,...,...,...,...,...
8997,195CDD,P195-R-DM-MLO,MLO,R,0
8998,196CDD,P196-L-DM-CC,CC,L,0
8999,196CDD,P196-L-DM-MLO,MLO,L,0
9000,202CDD,P202-R-DM-CC,CC,R,0


In [96]:
nocancer_val = nocancer[nocancer.patient_id.isin(par[1120:1190])]
nocancer_val

,patient_id,image_id,view,laterality,cancer
4558,15331,1304105664,CC,R,0
4559,15331,1238876883,MLO,R,0
4560,15331,2057830038,CC,R,0
4561,15331,881622734,MLO,R,0
4562,15331,256259567,CC,L,0
...,...,...,...,...,...
50019,1560345de7d8277b170ca3842eae6c97,cc347ba58148ec4cdbaa6e43dcbca0ab,MLO,L,0
50640,154CDD,P154-R-DM-CC,CC,R,0
50641,154CDD,P154-R-DM-MLO,MLO,R,0
50644,155CDD,P155-R-DM-CC,CC,R,0


In [97]:
evtrain = Evaluate(nocancer_train, 'nocancer_train')
evtrain.show()

Dataset nocancer_train 
 Số lượng image: 4504 
 Số luong patient: 1120
So luong CC và MLO
CC     2252
MLO    2252
Name: view, dtype: int64
So luong L và R
view  laterality
CC    L             1126
      R             1126
MLO   L             1126
      R             1126
dtype: int64
So luong img cancer: 0


In [98]:
evval = Evaluate(nocancer_val, 'nocancer_val')
evval.show()

Dataset nocancer_val 
 Số lượng image: 290 
 Số luong patient: 70
So luong CC và MLO
CC     145
MLO    145
Name: view, dtype: int64
So luong L và R
view  laterality
CC    L             71
      R             74
MLO   L             71
      R             74
dtype: int64
So luong img cancer: 0


In [77]:
290 + 4504 

4794

In [78]:
train_valNo = pd.concat([nocancer_train.image_id, nocancer_val.image_id]).values
train_valNo

array(['1874946579', '1864590858', '462822612', ..., 'P154-R-DM-MLO',
       'P155-R-DM-CC', 'P155-R-DM-MLO'], dtype=object)

In [82]:
len(train_valNo)

4794

In [83]:
train_valNo

array(['1874946579', '1864590858', '462822612', ..., 'P154-R-DM-MLO',
       'P155-R-DM-CC', 'P155-R-DM-MLO'], dtype=object)

In [84]:
len(nocancer) - 4794

44696

In [85]:
nouse = nocancer[~nocancer.image_id.isin(train_valNo)].reset_index()

In [79]:
nouse.to_csv(r'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Data_main\CSV_MAIN\no_use.csv')

In [86]:
train = pd.concat([cancer_train, nocancer_train]).reset_index().drop('index', axis=1)
train

,patient_id,image_id,view,laterality,cancer
0,10130,1360338805,CC,L,1
1,10130,1672636630,MLO,L,1
2,10130,613462606,CC,L,1
3,10130,388811999,MLO,L,1
4,10226,530620473,CC,L,1
...,...,...,...,...,...
8501,146CDD,P146-L-DM-MLO,MLO,L,0
8502,149CDD,P149-L-DM-CC,CC,L,0
8503,149CDD,P149-L-DM-MLO,MLO,L,0
8504,150CDD,P150-R-DM-CC,CC,R,0


In [88]:
val = pd.concat([cancer_val, nocancer_val]).reset_index().drop('index', axis=1)
val

,patient_id,image_id,view,laterality,cancer
0,D2-0617,D2-0617-1-4,CC,R,1
1,D2-0617,D2-0617-1-3,MLO,R,1
2,D2-0618,D2-0618-1-2,CC,L,1
3,D2-0618,D2-0618-1-1,MLO,L,1
4,D2-0619,D2-0619-1-4,CC,R,1
...,...,...,...,...,...
553,1560345de7d8277b170ca3842eae6c97,cc347ba58148ec4cdbaa6e43dcbca0ab,MLO,L,0
554,154CDD,P154-R-DM-CC,CC,R,0
555,154CDD,P154-R-DM-MLO,MLO,R,0
556,155CDD,P155-R-DM-CC,CC,R,0


In [90]:
ev = Evaluate(train, "Train")
ev.show()

Dataset Train 
 Số lượng image: 8506 
 Số luong patient: 3007
So luong CC và MLO
CC     4253
MLO    4253
Name: view, dtype: int64
So luong L và R
view  laterality
CC    L             2191
      R             2062
MLO   L             2191
      R             2062
dtype: int64
So luong img cancer: 4002


In [91]:
ev = Evaluate(val, "Train")
ev.show()

Dataset Train 
 Số lượng image: 558 
 Số luong patient: 203
So luong CC và MLO
CC     279
MLO    279
Name: view, dtype: int64
So luong L và R
view  laterality
CC    L             137
      R             142
MLO   L             137
      R             142
dtype: int64
So luong img cancer: 268


In [92]:
train.to_csv(r'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Data_main\CSV_MAIN\train.csv')

In [93]:
val.to_csv(r'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Data_main\CSV_MAIN\val.csv')

In [116]:
meta

,patient_id,image_id,view,laterality,cancer
0,10006,1874946579,CC,R,0
1,10006,1864590858,MLO,R,0
2,10006,462822612,CC,L,0
3,10006,1459541791,MLO,L,0
4,10011,541722628,CC,R,0
...,...,...,...,...,...
53745,D2-0747,D2-0747-1-3,MLO,R,1
53746,D2-0748,D2-0748-1-4,CC,R,1
53747,D2-0748,D2-0748-1-3,MLO,R,1
53748,D2-0749,D2-0749-1-2,CC,L,1


In [119]:
meta_x.cancer.value_counts()

0    49490
1     4260
Name: cancer, dtype: int64

In [120]:
meta_x.to_csv('D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Data_main\CSV_MAIN\meta_new.csv')

## Processing DDSM

In [4]:
ddsm = pd.read_csv(r"E:\WORKBASE\Project-rsna-breast-cancer-detection\Notebook_code\Cau_truc_data\meta_ddsm.csv").drop(['Unnamed: 0', 'index'], axis=1)
ddsm

,patient_id,image_id,view,laterality,cancer,fullPath
0,0001DDSM,0,CC,L,1,Cancer\0001\C_0001_1.LEFT_CC.jpg
1,0001DDSM,1,MLO,L,1,Cancer\0001\C_0001_1.LEFT_MLO.jpg
2,0001DDSM,2,CC,R,1,Cancer\0001\C_0001_1.RIGHT_CC.jpg
3,0001DDSM,3,MLO,R,1,Cancer\0001\C_0001_1.RIGHT_MLO.jpg
4,0002DDSM,4,CC,L,1,Cancer\0002\C_0002_1.LEFT_CC.jpg
...,...,...,...,...,...,...
5119,4607DDSM,5119,MLO,R,0,Normal\4607\D_4607_1.RIGHT_MLO.jpg
5120,4608DDSM,5120,CC,L,0,Normal\4608\D_4608_1.LEFT_CC.jpg
5121,4608DDSM,5121,MLO,L,0,Normal\4608\D_4608_1.LEFT_MLO.jpg
5122,4608DDSM,5122,CC,R,0,Normal\4608\D_4608_1.RIGHT_CC.jpg


In [5]:
ddsm.fullPath =ddsm.fullPath.apply(lambda x: x.replace('jpg', 'png'))

In [6]:
ddsm

,patient_id,image_id,view,laterality,cancer,fullPath
0,0001DDSM,0,CC,L,1,Cancer\0001\C_0001_1.LEFT_CC.png
1,0001DDSM,1,MLO,L,1,Cancer\0001\C_0001_1.LEFT_MLO.png
2,0001DDSM,2,CC,R,1,Cancer\0001\C_0001_1.RIGHT_CC.png
3,0001DDSM,3,MLO,R,1,Cancer\0001\C_0001_1.RIGHT_MLO.png
4,0002DDSM,4,CC,L,1,Cancer\0002\C_0002_1.LEFT_CC.png
...,...,...,...,...,...,...
5119,4607DDSM,5119,MLO,R,0,Normal\4607\D_4607_1.RIGHT_MLO.png
5120,4608DDSM,5120,CC,L,0,Normal\4608\D_4608_1.LEFT_CC.png
5121,4608DDSM,5121,MLO,L,0,Normal\4608\D_4608_1.LEFT_MLO.png
5122,4608DDSM,5122,CC,R,0,Normal\4608\D_4608_1.RIGHT_CC.png


In [7]:
from Setting_data import  *

In [8]:
import numpy as np
import pandas as pd
import multiprocessing as mp
class Setting_data:
    def __init__(self, data):
        
        self.data= data[['patient_id', 'image_id', 'view', 'laterality','cancer']]
        
        self.id = 0
        self.par = 0
        self.group = None
        
        
        self.meta_pro = pd.DataFrame(columns=['patient_id', 'image_id', 'view', 'laterality', 'cancer'])
    def set_par(self, par):
        self.par = par
    ## Zip columns img and cancer
    def zip_idimg_cancer(self):
        # Tạo id_img_zip để tận dụng cancer để sort xử lý ưu tiênc
        '''
        Chú ý dữ liệu phải được resetindex trước kh xử lý 
        '''
        self.data['id_img_zip'] = None
        for i in range(len(self.data)):
            # print(f'{self.data.cancer[i]}_{self.data.image_id[i]}')
            self.data.id_img_zip[i] = f'{self.data.cancer[i]}_{self.data.image_id[i]}'
        # ## Nhóm dữ liệu lại để xử lý
        self.group = self.data.groupby(['patient_id', 'view', 'laterality', 'id_img_zip']).size()
        self.data = self.data[(self.data.view == 'CC') |(self.data.view == 'MLO')]
        
    
    # Hàm filter_for2x là hàm dùng để cân bằng dữ liệu
    def filter_for_2x(self, par, lat):
        
        A = list(self.group[par]['CC'][lat].index)
        B = list(self.group[par]['MLO'][lat].index)
        
        A = sorted(A, key=lambda x: x[0],  reverse=True)# ls[0][0]
        B = sorted(B, key=lambda x: x[0],  reverse=True)# ls[0][0]
        i = 0
        j = 0
        # print(A)
        # print(B)
        while i < len(A) and j < len(B):
            a = A[i]
            b = B[i]
            # print(f"Lay a: {a.split('_')} \n lay b:{a.split('_')}")
            cancer_a, a = a.split('_')
            cancer_b, b = b.split('_')
            # print(a, b)
            ## update data
            # return
            
            self.meta_pro.loc[self.id] = [par, a, 'CC', lat, int(cancer_a)]
            # return
            self.meta_pro.loc[self.id+1] = [par, b, 'MLO', lat, int(cancer_b)]
            
            self.id+=2
            i+=1
            j+=1
  
    def check_cancer(self, par, views, lat):
        try: 
            return len(self.group[par][views][lat].index) >0
        except:
            return False
    def filter_main(self, par, lat):
        # Kiểm tra nếu CC + lat và MLO + lat mà đều chứa hình ảnh true
        if self.check_cancer(par, 'CC', lat) and self.check_cancer(par, 'MLO', lat):
           self.filter_for_2x(par, lat=lat)

            
    def save_file(self, path):
        self.meta_pro.to_csv(path)
    def procee_par(self, par):
        ## Process img R
    
        self.filter_main(par, lat='R')

        ## Process img L
        self.filter_main(par, lat='L')
    def process_multi(self, par):
        # print(par)
        self.procee_par(par)
    def multi_process(self, list_par):
        self.zip_idimg_cancer()
        for i in list_par:
            self.procee_par(i)

    

In [9]:
st = Setting_data(ddsm)
st.multi_process(list(ddsm.patient_id.unique()))

C:\Users\nguye\AppData\Local\Temp\ipykernel_9932\226691407.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.id_img_zip[i] = f'{self.data.cancer[i]}_{self.data.image_id[i]}'
C:\Users\nguye\AppData\Local\Temp\ipykernel_9932\226691407.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.id_img_zip[i] = f'{self.data.cancer[i]}_{self.data.image_id[i]}'
C:\Users\nguye\AppData\Local\Temp\ipykernel_9932\226691407.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/

In [10]:
st.meta_pro

,patient_id,image_id,view,laterality,cancer
0,0001DDSM,2,CC,R,1
1,0001DDSM,3,MLO,R,1
2,0001DDSM,0,CC,L,1
3,0001DDSM,1,MLO,L,1
4,0002DDSM,6,CC,R,1
...,...,...,...,...,...
5119,4607DDSM,5117,MLO,L,0
5120,4608DDSM,5122,CC,R,0
5121,4608DDSM,5123,MLO,R,0
5122,4608DDSM,5120,CC,L,0


In [113]:
CC_L_2720.png

,patient_id,image_id,view,laterality,cancer,fullPath
0,0001DDSM,0,CC,L,1,Cancer\0001\C_0001_1.LEFT_CC.png
1,0001DDSM,1,MLO,L,1,Cancer\0001\C_0001_1.LEFT_MLO.png
2,0001DDSM,2,CC,R,1,Cancer\0001\C_0001_1.RIGHT_CC.png
3,0001DDSM,3,MLO,R,1,Cancer\0001\C_0001_1.RIGHT_MLO.png
4,0002DDSM,4,CC,L,1,Cancer\0002\C_0002_1.LEFT_CC.png
...,...,...,...,...,...,...
5119,4607DDSM,5119,MLO,R,0,Normal\4607\D_4607_1.RIGHT_MLO.png
5120,4608DDSM,5120,CC,L,0,Normal\4608\D_4608_1.LEFT_CC.png
5121,4608DDSM,5121,MLO,L,0,Normal\4608\D_4608_1.LEFT_MLO.png
5122,4608DDSM,5122,CC,R,0,Normal\4608\D_4608_1.RIGHT_CC.png


In [11]:
ev = Evaluate(ddsm, 'DDSM')
ev.show()

NameError: name 'Evaluate' is not defined

In [8]:
from remove_image_views import *

In [11]:
# xoa DĐSM

In [9]:
ls = glob.glob('E:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Data_main\Dataset_VIEW\*DDSM')

In [10]:
len(ddsm.patient_id.unique())

1137

In [11]:
len(ls)

306

In [12]:
for path in ls:
    shutil.rmtree(path)

In [13]:
mv = move_image(ddsm, crop=False)

In [14]:
mv.my_method(list(ddsm.index))

In [117]:
ddsm[ddsm.image_id == 2720]

,patient_id,image_id,view,laterality,cancer,fullPath
2720,0003DDSM,2720,CC,L,0,Normal\0003\A_0003_1.LEFT_CC.png


In [118]:
ddsm

,patient_id,image_id,view,laterality,cancer,fullPath
0,0001DDSM,0,CC,L,1,Cancer\0001\C_0001_1.LEFT_CC.png
1,0001DDSM,1,MLO,L,1,Cancer\0001\C_0001_1.LEFT_MLO.png
2,0001DDSM,2,CC,R,1,Cancer\0001\C_0001_1.RIGHT_CC.png
3,0001DDSM,3,MLO,R,1,Cancer\0001\C_0001_1.RIGHT_MLO.png
4,0002DDSM,4,CC,L,1,Cancer\0002\C_0002_1.LEFT_CC.png
...,...,...,...,...,...,...
5119,4607DDSM,5119,MLO,R,0,Normal\4607\D_4607_1.RIGHT_MLO.png
5120,4608DDSM,5120,CC,L,0,Normal\4608\D_4608_1.LEFT_CC.png
5121,4608DDSM,5121,MLO,L,0,Normal\4608\D_4608_1.LEFT_MLO.png
5122,4608DDSM,5122,CC,R,0,Normal\4608\D_4608_1.RIGHT_CC.png


In [119]:
ddsm.to_csv('ddsm.csv')

In [120]:
from move_ddsm import *

In [121]:
my_method(list(ddsm.index))

In [20]:
len(ddsm)

5124

## ADD CMMD MIstake

In [94]:
meta = pd.read_csv("D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\TheChineseMammographyDatabase\TheChineseMammographyDatabase\meta_main.csv")
meta

,Unnamed: 0,index,patient_id,image_id,laterality,cancer,path,view
0,0,0,D1-0001,D1-0001_1-1,R,0,D:/OneDrive - Industrial University of HoChiMi...,MLO
1,1,1,D1-0001,D1-0001_1-2,R,0,D:/OneDrive - Industrial University of HoChiMi...,CC
2,2,2,D1-0002,D1-0002_1-1,L,0,D:/OneDrive - Industrial University of HoChiMi...,MLO
3,3,3,D1-0002,D1-0002_1-2,L,0,D:/OneDrive - Industrial University of HoChiMi...,CC
4,4,4,D1-0003,D1-0003_1-1,L,0,D:/OneDrive - Industrial University of HoChiMi...,MLO
...,...,...,...,...,...,...,...,...
3736,3736,3736,D2-0747,D2-0747_1-4,R,1,D:/OneDrive - Industrial University of HoChiMi...,CC
3737,3737,3737,D2-0748,D2-0748_1-3,R,1,D:/OneDrive - Industrial University of HoChiMi...,MLO
3738,3738,3738,D2-0748,D2-0748_1-4,R,1,D:/OneDrive - Industrial University of HoChiMi...,CC
3739,3739,3739,D2-0749,D2-0749_1-1,L,1,D:/OneDrive - Industrial University of HoChiMi...,MLO


In [95]:
meta.image_id = meta.image_id.apply(lambda x: x.replace('_', '-'))

In [96]:
meta

,Unnamed: 0,index,patient_id,image_id,laterality,cancer,path,view
0,0,0,D1-0001,D1-0001-1-1,R,0,D:/OneDrive - Industrial University of HoChiMi...,MLO
1,1,1,D1-0001,D1-0001-1-2,R,0,D:/OneDrive - Industrial University of HoChiMi...,CC
2,2,2,D1-0002,D1-0002-1-1,L,0,D:/OneDrive - Industrial University of HoChiMi...,MLO
3,3,3,D1-0002,D1-0002-1-2,L,0,D:/OneDrive - Industrial University of HoChiMi...,CC
4,4,4,D1-0003,D1-0003-1-1,L,0,D:/OneDrive - Industrial University of HoChiMi...,MLO
...,...,...,...,...,...,...,...,...
3736,3736,3736,D2-0747,D2-0747-1-4,R,1,D:/OneDrive - Industrial University of HoChiMi...,CC
3737,3737,3737,D2-0748,D2-0748-1-3,R,1,D:/OneDrive - Industrial University of HoChiMi...,MLO
3738,3738,3738,D2-0748,D2-0748-1-4,R,1,D:/OneDrive - Industrial University of HoChiMi...,CC
3739,3739,3739,D2-0749,D2-0749-1-1,L,1,D:/OneDrive - Industrial University of HoChiMi...,MLO


In [99]:
dict_path = {}
for i in range(len(meta)):
    dict_path[meta.image_id[i]] = meta.path[i]

In [100]:
dict_path

{'D1-0001-1-1': 'D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/TheChineseMammographyDatabase/TheChineseMammographyDatabase/CMMD/D1-0001/07-18-2010-NA-NA-79377/1.000000-NA-70244\\1-1.dcm',
 'D1-0001-1-2': 'D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/TheChineseMammographyDatabase/TheChineseMammographyDatabase/CMMD/D1-0001/07-18-2010-NA-NA-79377/1.000000-NA-70244\\1-2.dcm',
 'D1-0002-1-1': 'D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/TheChineseMammographyDatabase/TheChineseMammographyDatabase/CMMD/D1-0002/07-18-2010-NA-NA-49231/1.000000-NA-45313\\1-1.dcm',
 'D1-0002-1-2': 'D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/TheChineseMammographyDatabase/TheChineseMammographyDatabase/CMMD/D1-0002/07-18-2010-NA-NA-49231/1.000000-NA-45313\\1-2.dcm',
 'D1-0003-1-1': 'D:/OneDrive - I

In [ ]:
"D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Data_main\Dataset_VIEW\{id_imae\CC\L\CC_L_11ae77d09d90780bb56ed52bcefb4272.png"

In [ ]:
patient_id -->view --> lat --> 

In [101]:
cmmd = pd.read_csv('D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\DATA_STANDARD\CMMD.csv')
cmmd

,Unnamed: 0,patient_id,image_id,view,laterality,cancer
0,0,D1-0001,D1-0001-1-2,CC,R,0
1,1,D1-0001,D1-0001-1-1,MLO,R,0
2,2,D1-0002,D1-0002-1-2,CC,L,0
3,3,D1-0002,D1-0002-1-1,MLO,L,0
4,4,D1-0003,D1-0003-1-2,CC,L,0
...,...,...,...,...,...,...
3733,3733,D2-0747,D2-0747-1-3,MLO,R,1
3734,3734,D2-0748,D2-0748-1-4,CC,R,1
3735,3735,D2-0748,D2-0748-1-3,MLO,R,1
3736,3736,D2-0749,D2-0749-1-2,CC,L,1


In [102]:
dict_path

{'D1-0001-1-1': 'D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/TheChineseMammographyDatabase/TheChineseMammographyDatabase/CMMD/D1-0001/07-18-2010-NA-NA-79377/1.000000-NA-70244\\1-1.dcm',
 'D1-0001-1-2': 'D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/TheChineseMammographyDatabase/TheChineseMammographyDatabase/CMMD/D1-0001/07-18-2010-NA-NA-79377/1.000000-NA-70244\\1-2.dcm',
 'D1-0002-1-1': 'D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/TheChineseMammographyDatabase/TheChineseMammographyDatabase/CMMD/D1-0002/07-18-2010-NA-NA-49231/1.000000-NA-45313\\1-1.dcm',
 'D1-0002-1-2': 'D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/TheChineseMammographyDatabase/TheChineseMammographyDatabase/CMMD/D1-0002/07-18-2010-NA-NA-49231/1.000000-NA-45313\\1-2.dcm',
 'D1-0003-1-1': 'D:/OneDrive - I

In [103]:
path_misk = []

In [104]:
def procees_dcm(path):
    
    dicom = pydicom.dcmread(path)
    # try : 
    img = dicom.pixel_array
    # except:
    #     # print(get_info())
        
    #     f.write(self.get_info())
    #     return 
        
    img = apply_windowing(img, dicom)
    img = (img - img.min()) / (img.max() - img.min())

    if dicom.PhotometricInterpretation == "MONOCHROME1":  
        img = 1 - img
    image = np.uint8(img * 255)
    return image

In [105]:
from drop_foryou import *

In [106]:
path_misk

[]

In [109]:
for i in range(len(cmmd)):
    path = f"D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Data_main\Dataset_VIEW\{cmmd.patient_id[i]}\{cmmd.view[i]}\{cmmd.laterality[i]}\{cmmd.view[i]}_{cmmd.laterality[i]}_{cmmd.image_id[i]}.png"
    # print(path)
    id_img = cmmd.image_id[i]
    par = cmmd.patient_id[i]
    lat = cmmd.laterality[i]
    view = cmmd.view[i]
    if os.path.exists(path) == False:
        # path_misk.append(path)
        img = procees_dcm(dict_path[id_img])
        img = process_crop(img)
        # print(int(path_misk[int(id_img)]))
        # print(img)
        if os.path.exists(f"D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/Data_main/Dataset_VIEW/{par}/") == False:
            os.mkdir(f"D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/Data_main/Dataset_VIEW/{par}/")
        if os.path.exists(f"D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/Data_main/Dataset_VIEW/{par}/{view}/") == False:
            os.mkdir(f"D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/Data_main/Dataset_VIEW/{par}/{view}/")
        if os.path.exists(f"D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/Data_main/Dataset_VIEW/{par}/{view}/{lat}/") == False:
            os.mkdir(f"D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/Data_main/Dataset_VIEW/{par}/{view}/{lat}/")
        imageio.imwrite(path, img)
    

In [82]:
len(path_misk)

1046